# Байесовский поиск гиперпараметров с Optuna

В этом ноутбуке выполняется подбор гиперпараметров для модели CatBoost с использованием библиотеки Optuna и алгоритма TPE (Tree-structured Parzen Estimator).

In [ ]:
import optuna
import os
import mlflow
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import confusion_matrix, roc_auc_score, precision_score, recall_score, f1_score, log_loss
from collections import defaultdict
from dotenv import load_dotenv
import numpy as np
import pandas as pd
import psycopg
from sklearn.preprocessing import LabelEncoder
import warnings
import datetime
warnings.filterwarnings('ignore')

In [ ]:
# Загрузка переменных окружения
load_dotenv('../.env')

# Установка переменных окружения
TRACKING_SERVER_HOST = "localhost"
TRACKING_SERVER_PORT = 5001

# Настройка MLflow
mlflow_tracking_uri = f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}"
mlflow.set_tracking_uri(mlflow_tracking_uri)
mlflow.set_registry_uri(mlflow_tracking_uri)

# Настройка S3
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "https://storage.yandexcloud.net"
os.environ["AWS_ACCESS_KEY_ID"] = os.getenv("S3_ACCESS_KEY", "")
os.environ["AWS_SECRET_ACCESS_KEY"] = os.getenv("S3_SECRET_KEY", "")

EXPERIMENT_NAME = "catboost_hyperparameter_tuning"
RUN_NAME = "model_bayesian_search"
STUDY_DB_NAME = "sqlite:///optuna_study.db"
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
STUDY_NAME = f"churn_model_{timestamp}"

print(f"MLflow Tracking URI: {mlflow.get_tracking_uri()}")
print(f"Optuna Study DB: {STUDY_DB_NAME}")

In [ ]:
# Загрузка данных
TABLE_NAME = "users_churn"
connection = {"sslmode": "require", "target_session_attrs": "read-write"}
postgres_credentials = {
    "host": os.getenv("DB_DESTINATION_HOST"),
    "port": os.getenv("DB_DESTINATION_PORT"),
    "dbname": os.getenv("DB_DESTINATION_NAME"),
    "user": os.getenv("DB_DESTINATION_USER"),
    "password": os.getenv("DB_DESTINATION_PASSWORD"),
}
connection.update(postgres_credentials)

with psycopg.connect(**connection) as conn:
    with conn.cursor() as cur:
        cur.execute(f"SELECT * FROM {TABLE_NAME}")
        data = cur.fetchall()
        columns = [col[0] for col in cur.description]

df = pd.DataFrame(data, columns=columns)
print(f"✅ Данные загружены: {df.shape}")

In [ ]:
# Подготовка данных
columns_to_drop = ['id', 'customer_id', 'begin_date', 'end_date']
df_clean = df.drop(columns=columns_to_drop)

# Преобразуем категориальные признаки
label_encoders = {}
categorical_columns = df_clean.select_dtypes(include=['object']).columns

for col in categorical_columns:
    le = LabelEncoder()
    df_clean[col] = le.fit_transform(df_clean[col].astype(str))
    label_encoders[col] = le

# Разделение на признаки и целевую переменную
X = df_clean.drop('target', axis=1)
y = df_clean['target']

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42,
    stratify=y
)

print(f"✅ X_train: {X_train.shape}")
print(f"✅ X_test: {X_test.shape}")

In [ ]:
# Глобальные переменные для parent run
parent_run_id = None
experiment_id = None

def objective(trial: optuna.Trial) -> float:
    global parent_run_id, experiment_id
    
    param = {
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.1, log=True),
        "depth": trial.suggest_int("depth", 1, 12),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 0.1, 5),
        "random_strength": trial.suggest_float("random_strength", 0.1, 5),
        "loss_function": "Logloss",
        "task_type": "CPU",
        "random_seed": 0,
        "iterations": 300,
        "verbose": False,
    }
    
    skf = StratifiedKFold(n_splits=2)
    metrics = defaultdict(list)
    
    for train_index, val_index in skf.split(X_train, y_train):
        train_x = X_train.iloc[train_index]
        train_y = y_train.iloc[train_index]
        val_x = X_train.iloc[val_index]
        val_y = y_train.iloc[val_index]
        
        model = CatBoostClassifier(**param)
        model.fit(train_x, train_y)
        
        prediction = model.predict(val_x)
        probas = model.predict_proba(val_x)[:, 1]
        
        cm = confusion_matrix(val_y, prediction, normalize='all')
        tn, fp, fn, tp = cm.ravel()
        
        err1 = fp
        err2 = fn
        
        auc = roc_auc_score(val_y, probas)
        precision = precision_score(val_y, prediction)
        recall = recall_score(val_y, prediction)
        f1 = f1_score(val_y, prediction)
        logloss = log_loss(val_y, probas)
        
        metrics["err1"].append(err1)
        metrics["err2"].append(err2)
        metrics["auc"].append(auc)
        metrics["precision"].append(precision)
        metrics["recall"].append(recall)
        metrics["f1"].append(f1)
        metrics["logloss"].append(logloss)
    
    # Агрегированные метрики
    err_1 = np.median(np.array(metrics['err1']))
    err_2 = np.median(np.array(metrics['err2']))
    auc = np.median(np.array(metrics['auc']))
    precision = np.median(np.array(metrics['precision']))
    recall = np.median(np.array(metrics['recall']))
    f1 = np.median(np.array(metrics['f1']))
    logloss_val = np.median(np.array(metrics['logloss']))
    
    # Создаем child run для этого trial
    with mlflow.start_run(experiment_id=experiment_id, nested=True) as child_run:
        # Устанавливаем тег с parent run id
        mlflow.set_tag("mlflow.parentRunId", parent_run_id)
        
        # Логируем параметры
        mlflow.log_params(param)
        
        # Логируем метрики
        mlflow.log_metrics({
            "err1": err_1,
            "err2": err_2,
            "auc": auc,
            "precision": precision,
            "recall": recall,
            "f1": f1,
            "logloss": logloss_val
        })
    
    return auc

In [ ]:
# Создание/получение эксперимента
mlflow.set_experiment(EXPERIMENT_NAME)

print("Запуск байесовского поиска гиперпараметров...")

# Получаем experiment_id
experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
experiment_id = experiment.experiment_id

# Создание study
study = optuna.create_study(
    study_name=STUDY_NAME,
    storage=STUDY_DB_NAME,
    direction="maximize",
    sampler=optuna.samplers.TPESampler(),
    load_if_exists=True
)

# Основной блок оптимизации - создаем parent run
with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as parent_run:
    parent_run_id = parent_run.info.run_id
    print(f"\nParent MLflow Run ID: {parent_run_id}")
    
    # Оптимизация с 10 триалами
    print(f"\n🔍 Запуск оптимизации с 10 trials...")
    study.optimize(objective, n_trials=10)
    
    # Лучшие параметры
    best_params = study.best_params
    best_auc = study.best_value
    
    print(f"\n✅ Оптимизация завершена!")
    print(f"Лучший AUC: {best_auc:.4f}")
    print(f"Лучшие параметры: {best_params}")
    
    # Обучение финальной модели на всех train данных
    final_params = {
        **best_params,
        "loss_function": "Logloss",
        "task_type": "CPU", 
        "random_seed": 0,
        "iterations": 300,
        "verbose": False
    }
    
    best_model = CatBoostClassifier(**final_params)
    best_model.fit(X_train, y_train)
    
    # Логирование лучшей модели в parent run с артефактом 'cv'
    mlflow.catboost.log_model(best_model, "cv")
    
    # Логирование финальных метрик на test данных
    test_proba = best_model.predict_proba(X_test)[:, 1]
    test_pred = best_model.predict(X_test)
    
    # Расчёт метрик на тестовой выборке
    cm_test = confusion_matrix(y_test, test_pred, normalize='all')
    tn_test, fp_test, fn_test, tp_test = cm_test.ravel()
    
    test_auc = roc_auc_score(y_test, test_proba)
    test_precision = precision_score(y_test, test_pred)
    test_recall = recall_score(y_test, test_pred)
    test_f1 = f1_score(y_test, test_pred)
    test_logloss = log_loss(y_test, test_proba)
    
    # Логирование всех метрик в parent run
    mlflow.log_metric("test_auc", test_auc)
    mlflow.log_metric("test_precision", test_precision)
    mlflow.log_metric("test_recall", test_recall)
    mlflow.log_metric("test_f1", test_f1)
    mlflow.log_metric("test_logloss", test_logloss)
    mlflow.log_metric("test_err1", fp_test)
    mlflow.log_metric("test_err2", fn_test)
    
    # Логирование CV метрик
    mlflow.log_metric("cv_best_auc", best_auc)
    
    # Логирование лучших параметров
    mlflow.log_params(best_params)
    
    # Логируем дополнительные параметры
    mlflow.log_param("iterations", 300)
    mlflow.log_param("loss_function", "Logloss")
    mlflow.log_param("task_type", "CPU")
    mlflow.log_param("random_seed", 0)
    mlflow.log_param("test_size", 0.2)
    mlflow.log_param("cv_folds", 2)
    mlflow.log_param("search_method", "TPESampler")
    mlflow.log_param("n_trials", 10)
    
    print("\n✅ Лучшая модель залогирована в MLflow!")
    print(f"Parent Run ID: {parent_run.info.run_id}")
    print(f"\n📊 Метрики на тестовой выборке:")
    print(f"   ROC-AUC: {test_auc:.4f}")
    print(f"   Precision: {test_precision:.4f}")
    print(f"   Recall: {test_recall:.4f}")
    print(f"   F1-Score: {test_f1:.4f}")

print("\n" + "="*70)
print("✅ ЗАВЕРШЕНО!")
print("="*70)
print(f"Количество завершенных триалов: {len(study.trials)}")
print(f"Лучший CV AUC: {study.best_value:.4f}")
print(f"Лучшие параметры: {study.best_params}")
print(f"\nParent Run ID: {parent_run_id}")

In [ ]:
# Проверка child runs
filter_string = f"tags.mlflow.parentRunId = '{parent_run_id}'"
child_runs = mlflow.search_runs(
    experiment_ids=[experiment_id],
    filter_string=filter_string
)

print("="*70)
print(f"ПРОВЕРКА CHILD RUNS")
print("="*70)
print(f"Parent Run ID: {parent_run_id}")
print(f"Найдено child runs: {len(child_runs)}")

if len(child_runs) == 10:
    print("✅ Найдено 10 child runs")
else:
    print(f"⚠️ Ожидалось 10 child runs, найдено {len(child_runs)}")